In [1]:
import os

import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks.tiled_inference import TiledInferenceParameters

from implementation.training.utils import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.bag3d import BAG3DTileStore
from roofsense.inference.tile import TiledInferenceEngine
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

C:\Documents\RoofSense\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.5 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
study_name = "optim"
optim_log_dirpath = os.path.join(r"C:\Documents\RoofSense\logs\3dgeoinfo", study_name)

study = optuna.load_study(
    study_name="optim", storage=f"sqlite:///{optim_log_dirpath}/storage.db"
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

config = TrainingTaskHyperparameterTuningConfig(
    # Add constant settings.
    # Encoder
    encoder="tu-resnet18d",
    zero_init_last=True,
    # Loss
    label_smoothing=0.1,
    # Optimizer
    optimizer="AdamW",
    # LR Scheduler
    lr_scheduler="CosineAnnealingLR",
    **best_params,
)

In [3]:
# Quantitative Performance Evaluation
torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

model = create_model(config)
model_path = os.path.join(
    optim_log_dirpath, f"version_{study.best_trial.number}", "ckpts", "best.ckpt"
)

task: TrainingTask = TrainingTask.load_from_checkpoint(
    checkpoint_path=model_path, model=model
)

datamodule = TrainingDataModule(
    root=r"C:\Documents\RoofSense\roofsense\dataset",
    append_lab=config.append_lab,
    append_tgi=config.append_tgi,
)

test_log_dirpath = os.path.join(os.path.dirname(optim_log_dirpath), "test")
trainer = Trainer(
    logger=TensorBoardLogger(save_dir=test_log_dirpath, name="quantitative"),
    benchmark=True,
)
trainer.test(task, datamodule=datamodule)

Seed set to 0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18d.ra2_in1k)
INFO:timm.models._hub:[timm/resnet18d.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv1.0 pretrained weights from 3 to 10 channel(s)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:05<00:00,  1.01it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9483334422111511     │
│       tst/Accuracy2       │    0.8826488256454468     │
│       tst/Accuracy3       │    0.9835857152938843     │
│       tst/Accuracy4       │     0.92097008228302      │
│       tst/Accuracy5       │    0.5459764003753662     │
│       tst/Accuracy6       │    0.9796044230461121     │
│       tst/Accuracy7       │    0.7573438286781311     │
│       tst/Accuracy8       │    0.7807404398918152     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │     0.92611163854599      │
│       tst/F1Score2        │    0.9207127690315247     │
│       tst/F1Score3        │     0.959421694278717     │
│       tst/F1Score4        │     0.928403377532959     │
│       tst/F1Score5        │    0.5604041218757629     │
│       tst/F1Score6        │    0.9317227005958557     │
│       tst/F1Score7        │    0.7900195717811584     │
│       tst/F1Score8        │    0.7189390659332275     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │    0.8623909950256348     │
│     tst/JaccardIndex2     │    0.8530747890472412     │
│     tst/JaccardIndex3     │    0.9220080971717834     │
│     tst/JaccardIndex4     │    0.8663738965988159     │
│     tst/JaccardIndex5     │     0.389278769493103     │
│     tst/JaccardIndex6     │    0.8721730709075928     │
│     tst/JaccardIndex7     │    0.6529192924499512     │
│     tst/JaccardIndex8     │    0.5612059831619263     │
│         tst/Loss          │     2.000236749649048     │
│     tst/MacroAccuracy     │    0.8499003648757935     │
│     tst/MacroF1Score      │    0.8419668674468994     │
│   tst/MacroJaccardIndex   │    0.7474281191825867     │
│    tst/MacroPrecision     │    0.8369067311286926     │
│      tst/MacroRecall      │    0.8499003648757935     │
│     tst/MicroAccuracy     │    0.9113106727600098     │
│     tst/MicroF1Score      │    0.9113106727600098     │
│   tst/MicroJaccardIndex   │    0.8371540307998657     │
│    tst/MicroPrecision     │    0.9113106727600098     │
│      tst/MicroRecall      │    0.9113106727600098     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9049074649810791     │
│      tst/Precision2       │    0.9622076153755188     │
│      tst/Precision3       │    0.9364164471626282     │
│      tst/Precision4       │    0.9359576106071472     │
│      tst/Precision5       │    0.5756150484085083     │
│      tst/Precision6       │    0.8883035778999329     │
│      tst/Precision7       │    0.8256420493125916     │
│      tst/Precision8       │    0.6662040948867798     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9483334422111511     │
│        tst/Recall2        │    0.8826488256454468     │
│        tst/Recall3        │    0.9835857152938843     │
│        tst/Recall4        │     0.92097008228302      │
│        tst/Recall5        │    0.5459764003753662     │
│        tst/Recall6        │    0.9796044230461121     │
│        tst/Recall7        │    0.7573438286781311     │
│        tst/Recall8        │    0.7807404398918152     │
└───────────────────────────┴───────────────────────────┘

[{'tst/Loss': 2.000236749649048,
  'tst/MacroAccuracy': 0.8499003648757935,
  'tst/MacroF1Score': 0.8419668674468994,
  'tst/MacroJaccardIndex': 0.7474281191825867,
  'tst/MacroPrecision': 0.8369067311286926,
  'tst/MacroRecall': 0.8499003648757935,
  'tst/MicroAccuracy': 0.9113106727600098,
  'tst/MicroF1Score': 0.9113106727600098,
  'tst/MicroJaccardIndex': 0.8371540307998657,
  'tst/MicroPrecision': 0.9113106727600098,
  'tst/MicroRecall': 0.9113106727600098,
  'tst/Accuracy0': 0.0,
  'tst/Accuracy1': 0.9483334422111511,
  'tst/Accuracy2': 0.8826488256454468,
  'tst/Accuracy3': 0.9835857152938843,
  'tst/Accuracy4': 0.92097008228302,
  'tst/Accuracy5': 0.5459764003753662,
  'tst/Accuracy6': 0.9796044230461121,
  'tst/Accuracy7': 0.7573438286781311,
  'tst/Accuracy8': 0.7807404398918152,
  'tst/F1Score0': 0.0,
  'tst/F1Score1': 0.92611163854599,
  'tst/F1Score2': 0.9207127690315247,
  'tst/F1Score3': 0.959421694278717,
  'tst/F1Score4': 0.928403377532959,
  'tst/F1Score5': 0.56040412

In [4]:
# Qualitative Performance Evaluation
tile_id = "9-284-556"

TiledInferenceEngine(
    ckpt_path=model_path, tile_store=BAG3DTileStore(), model=model
).run(
    tile_id=tile_id,
    dst_filepath=os.path.join(
        test_log_dirpath, "qualitative", f"{tile_id}.map.pixel.tif"
    ),
    params=TiledInferenceParameters(h_crop=512, h_stride=256, w_crop=512, w_stride=256),
)

Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Documents\RoofSense\roofsense\utilities\file.py:70: UserWarning: The specified path: 'C:\\Documents\\RoofSense\\logs\\3dgeoinfo\\test\\qualitative\\9-284-556.map.pixel.tif' does not exist in the system. The value of the 'overwrite' flag will be ignored.
  warnings.warn(msg, UserWarning)


ValueError: Found no raster stack corresponding to tile with ID: 9-284-556.